# [백준/세상의 중심에서...](https://www.acmicpc.net/problem/2389)


## 풀이과정


### 첫번째 시도


#### 풀이과정


In [ ]:
# refer https://www.acmicpc.net/source/55045393


def cross(a: complex, b: complex) -> float:
    return (a.conjugate() * b).imag


def ccw(a: complex, b: complex, c: complex):
    return cross((a - c), (b - c)) > 0


def key(a: complex):
    return a.real, a.imag


def get_center_2(a: complex, b: complex) -> complex:
    return (a + b) / 2


def get_center_3(a: complex, b: complex, c: complex) -> complex:
    ab = b - a
    ac = c - a
    return a + (abs(ac) ** 2 * ab - abs(ab) ** 2 * ac) * 1j / (2 * cross(ab, ac))


def get_circle(bound: list[complex]):
    center = (get_center_3 if len(bound) > 2 else get_center_2)(*bound)
    return center, abs(center - bound[0])


def init(bound: list[complex]) -> tuple[complex, float]:
    return get_circle(bound) if len(bound) > 1 else ((bound[0] if bound else 0j), 0.0)


def enclosed(bound: list[complex], points: list[complex]) -> tuple[complex, float]:
    if len(bound) == 3:
        return get_circle(bound)
    center, radius = init(bound)
    for i, p in enumerate(points):
        if abs(center - p) > radius:
            center, radius = enclosed(bound + [p], points[:i])
    return center, radius


def solution():
    import sys

    inputs = sys.stdin.read().strip().split("\n")[1:]
    points = sorted(set(complex(*map(float, line.split())) for line in inputs), key=key)
    center, radius = enclosed([], points)
    print(center.real, center.imag, radius)


solution()

## 해답


In [1]:
def cross(a: complex, b: complex) -> float:
    return (a.conjugate() * b).imag

In [2]:
def ccw(a: complex, b: complex, c: complex):
    return cross((a - c), (b - c)) > 0

In [3]:
def key(a: complex):
    return a.real, a.imag

In [4]:
def get_center_2(a: complex, b: complex) -> complex:
    return (a + b) / 2

In [5]:
def get_center_3(a: complex, b: complex, c: complex) -> complex:
    ab = b - a
    ac = c - a
    return a + (abs(ac) ** 2 * ab - abs(ab) ** 2 * ac) * 1j / (2 * cross(ab, ac))

In [6]:
def get_circle(bound: list[complex]):
    center = (get_center_3 if len(bound) > 2 else get_center_2)(*bound)
    return center, abs(center - bound[0])

In [7]:
def init(bound: list[complex]) -> tuple[complex, float]:
    return get_circle(bound) if len(bound) > 1 else ((bound[0] if bound else 0j), 0.0)

In [8]:
def enclosed(bound: list[complex], points: list[complex]) -> tuple[complex, float]:
    if len(bound) == 3:
        return get_circle(bound)
    center, radius = init(bound)
    for i, p in enumerate(points):
        if abs(center - p) > radius:
            center, radius = enclosed(bound + [p], points[:i])
    return center, radius

In [9]:
def solution():
    import sys

    inputs = sys.stdin.read().strip().split("\n")[1:]
    points = sorted(set(complex(*map(float, line.split())) for line in inputs), key=key)
    center, radius = enclosed([], points)
    print(center.real, center.imag, radius)

## 예제


In [10]:
# 백준 문제 풀이용 예제 실행 코드
from bwj import test

test_solution = test(solution)

# test_solution("""""")
# test_solution(read("fn").read())

In [11]:
test_solution(
    """3
1 1
2 2
3 3
"""
)  # 2 2 1.4142135624

2.0 2.0 1.4142135623730951
